### Anomoly Detection: Codeup Curriculum Logs

    Created By: Chen Chen Feng & Mijail Mariano

    Data Science, Kalpana Cohort

    25AUGUST2022

----

In [ ]:
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300

# diasbling warning messages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# visualization libraries/modules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# prepare module for data cleaning/manipulation
import prepare_final

# acquire module for acquiring the Codeup curriculum logs dataset
import acquire_final
from acquire_final import

# skimpy module to clean column names
from skimpy import clean_columns

# regular expression module
import re

import env_mm
from env_mm import user, password, host, get_connection